In [5]:
import pandas as pd
import numpy as np
import requests
import re
import time
from bs4 import BeautifulSoup as bs
from dbio4 import to_db, load_data

# 뉴스목록, 날짜, id수집

In [ ]:
url1 = "https://fintech.or.kr/web/board/boardContentsList.do"

In [ ]:
payload = dict(miv_pageNo=1,mode="W",board_id=6)

In [ ]:
r1 = requests.get(url1, params=payload)
print(r1.status_code)
soup = bs(r1.content, "lxml")
soup

In [ ]:
https://fintech.or.kr/web/board/boardContentsView.do

In [ ]:
miv_pageNo=1&miv_pageSize=&total_cnt=&LISTOP=&mode=W&contents_id=39547f2a0ee34a5097ff4e92f6df9f6e&board_id=6&p_reg_userno=&searchkey=I&searchtxt=

mobile용

In [ ]:
len(soup.select("ul.list > li > a"))

Web용은 Table 안에

In [ ]:
#제목과 뉴스id
soup.select("a.txtl")

In [ ]:
#제목과 뉴스id
soup.select("a.txtl")[0]['href'].split("'")[1]

In [ ]:
#날짜
soup.select("td.last")[0].text.strip()

# news list id와 날짜 모으기

In [ ]:
contents_id_list = []
for date, content_list in zip(soup.select("td.last"), soup.select("a.txtl")):
    date = date.text.strip()
    contents_id = content_list['href'].split("'")[1]
    contents_id_list.append((date, contents_id))
contents_id_list

In [ ]:
#news list의 url
url2 = "https://fintech.or.kr/web/board/boardContentsView.do"

In [ ]:
#news list의 payload
payload2 = dict(miv_pageNo=1,mode="W",contents_id="39547f2a0ee34a5097ff4e92f6df9f6e",board_id=6,searchkey="I")

In [ ]:
r2 = requests.get(url2, params=payload2)
print(r2.status_code)
soup2 = bs(r2.content, 'lxml')
soup2

In [ ]:
# 뉴스 리스트 목록
soup2.select("tbody td a")[0].text

In [ ]:
soup2.select("tbody td a")[0]['href']

In [ ]:
soup2.select("tbody td a")[6]['href']

# 위에서 개별 작업한 코드 모으기

In [ ]:
# 핀테크 뉴스 리스트, 날짜, content_id 수집
contents_id_list = []
for page in range(1,20):
    url1 = "https://fintech.or.kr/web/board/boardContentsList.do"
    payload = dict(miv_pageNo=page, mode='W', board_id=6)
    r1 = requests.post(url1, data=payload)
    print(r1.status_code)
    soup = bs(r1.content, "lxml")
    
    for date, content_list in zip(soup.select("td.last"), soup.select("a.txtl")):
        date = date. text.strip()
        contents_id = content_list ['href'].split("'") [1]
        contents_id_list.append((date, contents_id))
    time.sleep(2)
contents_id_list
result = {}      
# 날짜별  뉴스 리스트의 세부 뉴스 제목, 링크
for idx, (date, content_id) in enumerate(contents_id_list):
    print(f"{idx+1}/{len(contents_id_list)} 수집중", end="\r")
    url2 = "https://fintech.or.kr/web/board/boardContentsView.do"
    payload2 = dict(miv_pageNo=1,mode="W",contents_id=content_id,board_id=6,searchkey="I")
    r2 = requests.get(url2, params=payload2)
    print(r2.status_code)
    soup2 = bs(r2.content, 'lxml')

    for td in soup2.select("tbody td a"):
        if "https://n.news.naver.com" in td['href'] or "https://fintech.or.kr/web/" in td['href']:
            result.setdefault('date', []).append(date)
            result.setdefault('제목', []).append(td.text)
            result.setdefault('뉴스링크', []).append(td['href'])

df = pd.DataFrame(result)

In [ ]:
df

# 세부링크로 들어가서 뉴스 본문 수집

In [ ]:
df2 = df[df['date'] > "2025-01-01"].copy()

In [ ]:
df2['뉴스링크'][6]

In [ ]:
# 핀테크 지원센터 글
r3 = requests.get(df2['뉴스링크'][0])
soup3 = bs(r3.content, 'lxml')
soup3.select_one("div.content").text

In [ ]:
# 네이버 뉴스 글
r3 = requests.get(df2['뉴스링크'][6])
soup3 = bs(r3.content, 'lxml')
soup3.select_one("#dic_area").text.strip()

In [ ]:
# 네이버 뉴스 글
r3 = requests.get(df2['뉴스링크'][12])
soup3 = bs(r3.content, 'lxml')
soup3.select_one("#dic_area").text.strip()

In [ ]:
df2['뉴스본문'] = ""
df2

In [7]:
for idx, link in enumerate(df2['뉴스링크'][:10]):
    print(f"{idx+1}/{len(df2['뉴스링크'][:10])} 수집중", end="\r")
    r4 = requests.get(link)
    soup4 = bs(r4.content, 'lxml')
    if "https://fintech.or.kr/web/" in link:
        r3 = requests.get(link)
        soup3 = bs(r3.content, 'lxml')
        article = soup3.select_one("div.content").text
        df2.loc[idx, '뉴스본문'] = article
    elif "https://n.news.naver.com" in link:
        r3 = requests.get(link)
        soup3 = bs(r3.content, 'lxml')
        article = soup3.select_one("#dic_area").text
        df2.loc[idx, '뉴스본문'] = article
    to_db("fintech_center", "news_articles", df2.loc[[idx], :])
    time.sleep(3)
df2

fintech_center 데이터베이스 확인/생성 완료
fintech_center 데이터베이스 확인/생성 완료
fintech_center 데이터베이스 확인/생성 완료
fintech_center 데이터베이스 확인/생성 완료
fintech_center 데이터베이스 확인/생성 완료
fintech_center 데이터베이스 확인/생성 완료
fintech_center 데이터베이스 확인/생성 완료
fintech_center 데이터베이스 확인/생성 완료
fintech_center 데이터베이스 확인/생성 완료
fintech_center 데이터베이스 확인/생성 완료


,date,제목,뉴스링크,뉴스본문
0,2025-11-26,[한국핀테크지원센터X구름] K-디지털트레이닝 '핀테크 인턴십 코스' 4기 훈련생 모...,https://fintech.or.kr/web/board/boardContentsV...,[한국핀테크지원센터x구름]「K-디지털 트레이닝 핀테크 인턴십 코스」4기 훈련생 모집...
1,2025-11-26,[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...,https://fintech.or.kr/web/board/boardContentsV...,"안녕하세요. 핀테크산업협회 사무국입니다.협회는 핀테크 기업과 금융기관, AFA 등 ..."
2,2025-11-26,[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...,https://fintech.or.kr/web/board/boardContentsV...,안녕하세요. 한국핀테크지원센터입니다.2025년 금융특화 계층별 AI역량강화 과정 ‘...
3,2025-11-26,[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...,https://fintech.or.kr/web/board/boardContentsV...,2025년‘핀테크기업 온라인 채용관’ (사람인 saramin) 참여 기업 모집 ...
4,2025-11-26,"""부산을 해양금융 허브로""…금감원·부산시·해진공·조선업계 총집결",https://n.news.naver.com/mnews/article/008/000...,\n\n\n\n\n부산 영도구 봉래산 일대금융감독원이 부산을 글로벌 해양금융허브로 ...
...,...,...,...,...
11272,2025-01-02,"""당장 돈 안되고 성능향상 기대 못 미쳐도""… 세계는 AI인프라 `영끌`",https://n.news.naver.com/mnews/article/029/000...,NaN
11273,2025-01-02,트럼프 2.0시대 중국…미 동맹국 이탈 기대하며 버티기[다시 만난 트럼프②],https://n.news.naver.com/mnews/article/032/000...,NaN
11274,2025-01-02,'美 51번째 주' 모욕 당했는데…트럼프에 찍소리 못하는 이유 [김리안의 에네르기파...,https://n.news.naver.com/mnews/article/015/000...,NaN
11275,2025-01-02,[산업은행] 2025년 상반기 KDB NextONE 참여 스타트업 모집(서울/부산)...,https://fintech.or.kr/web/board/boardContentsV...,NaN


In [ ]:
df2['뉴스본문'][5]

In [ ]:
#df2.to_csv("./data/fintech_center.csv")

In [2]:
df2 = pd.read_csv("./data/fintech_center.csv", index_col=0)
df2

,date,제목,뉴스링크,뉴스본문
0,2025-11-26,[한국핀테크지원센터X구름] K-디지털트레이닝 '핀테크 인턴십 코스' 4기 훈련생 모...,https://fintech.or.kr/web/board/boardContentsV...,[한국핀테크지원센터x구름]「K-디지털 트레이닝 핀테크 인턴십 코스」4기 훈련생 모집...
1,2025-11-26,[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...,https://fintech.or.kr/web/board/boardContentsV...,"안녕하세요. 핀테크산업협회 사무국입니다.협회는 핀테크 기업과 금융기관, AFA 등 ..."
2,2025-11-26,[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...,https://fintech.or.kr/web/board/boardContentsV...,안녕하세요. 한국핀테크지원센터입니다.2025년 금융특화 계층별 AI역량강화 과정 ‘...
3,2025-11-26,[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...,https://fintech.or.kr/web/board/boardContentsV...,2025년‘핀테크기업 온라인 채용관’ (사람인 saramin) 참여 기업 모집 ...
4,2025-11-26,"""부산을 해양금융 허브로""…금감원·부산시·해진공·조선업계 총집결",https://n.news.naver.com/mnews/article/008/000...,\n\n\n\n\n부산 영도구 봉래산 일대금융감독원이 부산을 글로벌 해양금융허브로 ...
...,...,...,...,...
11272,2025-01-02,"""당장 돈 안되고 성능향상 기대 못 미쳐도""… 세계는 AI인프라 `영끌`",https://n.news.naver.com/mnews/article/029/000...,NaN
11273,2025-01-02,트럼프 2.0시대 중국…미 동맹국 이탈 기대하며 버티기[다시 만난 트럼프②],https://n.news.naver.com/mnews/article/032/000...,NaN
11274,2025-01-02,'美 51번째 주' 모욕 당했는데…트럼프에 찍소리 못하는 이유 [김리안의 에네르기파...,https://n.news.naver.com/mnews/article/015/000...,NaN
11275,2025-01-02,[산업은행] 2025년 상반기 KDB NextONE 참여 스타트업 모집(서울/부산)...,https://fintech.or.kr/web/board/boardContentsV...,NaN


In [ ]:
#!pip install sqlalchemy pymysql

In [ ]:
#!pip install cryptography 

In [8]:
df3 = load_data("fintech_center", "news_articles")
df3

fintech_center 데이터베이스 확인/생성 완료


,date,제목,뉴스링크,뉴스본문
0,2025-11-26,[한국핀테크지원센터X구름] K-디지털트레이닝 '핀테크 인턴십 코스' 4기 훈련생 모...,https://fintech.or.kr/web/board/boardContentsV...,[한국핀테크지원센터x구름]「K-디지털 트레이닝 핀테크 인턴십 코스」4기 훈련생 모집...
1,2025-11-26,[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...,https://fintech.or.kr/web/board/boardContentsV...,"안녕하세요. 핀테크산업협회 사무국입니다.협회는 핀테크 기업과 금융기관, AFA 등 ..."
2,2025-11-26,[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...,https://fintech.or.kr/web/board/boardContentsV...,안녕하세요. 한국핀테크지원센터입니다.2025년 금융특화 계층별 AI역량강화 과정 ‘...
3,2025-11-26,[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...,https://fintech.or.kr/web/board/boardContentsV...,2025년‘핀테크기업 온라인 채용관’ (사람인 saramin) 참여 기업 모집 ...
4,2025-11-26,"""부산을 해양금융 허브로""…금감원·부산시·해진공·조선업계 총집결",https://n.news.naver.com/mnews/article/008/000...,\n\n\n\n\n부산 영도구 봉래산 일대금융감독원이 부산을 글로벌 해양금융허브로 ...
5,2025-11-26,"금융당국, 불공정거래 차단 위해 ‘조사-수사’ 유기적 연계… 제도 개선 논의",https://n.news.naver.com/mnews/article/366/000...,\n\t\t\t 금융당국은 합동대응단의 불공정 거래 조사와 수사기관의 ...
6,2025-11-26,"금감원 ""증권사 전산사고 엄정 검사·제재""…IT·정보보안 간담회",https://n.news.naver.com/mnews/article/003/001...,\n\n\n\n\n[서울=뉴시스]이지민 기자 = 증권사의 전산사고가 끊이지 않고 발...
7,2025-11-26,"산업계 찾은 금융위 부위원장 ""생산적 금융 대전환 책임지고 추진""",https://n.news.naver.com/mnews/article/421/000...,"\n기업인들, 자본 조달 어려움 호소…은행 기업대출 RAW 개선도 건의\n\n\n\..."
8,2025-11-26,"금융위, '역전세 반환대출' 규제완화 일몰 폐지…상시화 전환",https://n.news.naver.com/mnews/article/277/000...,\n\t\t\t금융당국이 올해 말 종료 예정이던 역전세 반환대출 규제 완화 조치의 ...
9,2025-11-26,"'내부통제 미흡' 대형 GA 30% 육박...""내년 우선검사 실시""",https://n.news.naver.com/mnews/article/014/000...,"\n2024년도 내부통제 실태 평가 규모 작을 수록 내부통제 미비금감원 ""평가 저조..."
